# Assignment 1

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [1]:
import matplotlib.dates as dates
import matplotlib.ticker as ticker

In [ ]:
#importing and inspecting data
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df.head()

# Converting to C° degrees
df['Data_Value'] = df['Data_Value'] * 0.1 

# Manipulating dates:
# Applying to_datetime method to the Date column to get rid of the datetime error
df['Date'] = list(map(pd.to_datetime, df['Date']))
# Splitting dates
df['Year'] = df['Date'].dt.year
df['Month_Day'] = df['Date'].dt.strftime('%m-%d')
# Filtering out leap days
df = df[df['Month_Day']!='02-29']

# Line data
# Y Axis: taking only the record high or record low temperature for each day across the requested time interval
df_2005_2014_max = df[ (df['Year'] >= 2005) & (df['Year'] <= 2014) & (df['Element'] == 'TMAX') ].groupby(by=["Month_Day"]).max().reset_index()[['Month_Day', 'Data_Value']].rename(columns={'Data_Value': 'Data_Value_max'})
df_2005_2014_min = df[ (df['Year'] >= 2005) & (df['Year'] <= 2014) & (df['Element'] == 'TMIN') ].groupby(by=["Month_Day"]).min().reset_index()[['Month_Day', 'Data_Value']].rename(columns={'Data_Value': 'Data_Value_min'})

# X Axis: taking dates from 2015 only
df_2015 = df[ (df['Date'] >= '2015-01-01') & (df['Date'] <= '2015-12-31')]['Date'].sort_values().unique()
df_2015 = list(map(pd.to_datetime, df_2015))


# Scatterplot data
# Y Axis: taking only 2015 temperatures which are breaking the previous decade record high or record low
df_2015_max = df[ (df['Year'] == 2015) & (df['Element'] == 'TMAX') ].groupby(by=["Month_Day"]).max().reset_index()[['Month_Day', 'Data_Value']].rename(columns={'Data_Value': 'Data_Value_max_2015', 'Month_Day': 'Month_Day_2015'})
df_scatter_max = pd.concat([df_2005_2014_max, df_2015_max], axis = 1)
df_scatter_max = df_scatter_max[(df_scatter_max['Data_Value_max_2015'] > df_scatter_max['Data_Value_max']) & (df_scatter_max['Month_Day'] == df_scatter_max['Month_Day_2015'])][['Month_Day_2015', 'Data_Value_max_2015']]

df_2015_min = df[ (df['Year'] == 2015) & (df['Element'] == 'TMIN') ].groupby(by=["Month_Day"]).min().reset_index()[['Month_Day', 'Data_Value']].rename(columns={'Data_Value': 'Data_Value_min_2015', 'Month_Day': 'Month_Day_2015'})
df_scatter_min = pd.concat([df_2005_2014_min, df_2015_min], axis = 1)
df_scatter_min = df_scatter_min[(df_scatter_min['Data_Value_min_2015'] < df_scatter_min['Data_Value_min']) & (df_scatter_min['Month_Day'] == df_scatter_min['Month_Day_2015'])][['Month_Day_2015', 'Data_Value_min_2015']]

# X axis: converting the month_day back to year_month_day to be consistent with the already existing x axis
df_scatter_max['Month_Day_2015'] = '2015-' + df_scatter_max['Month_Day_2015']
df_scatter_max['Month_Day_2015']  = list(map(pd.to_datetime, df_scatter_max['Month_Day_2015']))

df_scatter_min['Month_Day_2015'] = '2015-' + df_scatter_min['Month_Day_2015']
df_scatter_min['Month_Day_2015']  = list(map(pd.to_datetime, df_scatter_min['Month_Day_2015']))



# Plotting data
plt.figure()

plt.plot(df_2015, df_2005_2014_max['Data_Value_max'], '-', df_2015, df_2005_2014_min['Data_Value_min'], '-', linewidth=1)


# Adding the scatter plot
plt.scatter(df_scatter_max.Month_Day_2015.values, df_scatter_max.Data_Value_max_2015.values)
plt.scatter(df_scatter_min.Month_Day_2015.values, df_scatter_min.Data_Value_min_2015.values)

# Set x and y limits
ax = plt.gca()
ax.axis(['2015/01/01','2015/12/31',-50,50])

# Axis names and title
plt.xlabel('Date')
plt.ylabel('° Celsius')
plt.title('Record temperatures by day of year in Ann Arbour, Michigan (2005-2015)')

# Legend
plt.legend(['Record high (2005-2014)','Record low (2005-2014)','Record breaking high in 2015','Record breaking low in 2015'], frameon=False)


# Fill the area between the record high and record low temperatures
plt.gca().fill_between(df_2015, 
                       df_2005_2014_min['Data_Value_min'], df_2005_2014_max['Data_Value_max'], 
                       facecolor='grey', 
                       alpha=0.25)

# Setting the ticks
ax.xaxis.set_major_locator(dates.MonthLocator())
ax.xaxis.set_minor_locator(dates.MonthLocator(bymonthday=15))
ax.xaxis.set_major_formatter(ticker.NullFormatter())
ax.xaxis.set_minor_formatter(dates.DateFormatter('%b'))